# Save trivial to fermionic non-trivial models

Created 14/04/2025

Objectives:
* Take a hamiltonian as a linear combination of the trivial fermionic model and a  model trivial in proj rep but non-trivial in fermionic decoration. Sweep over linear combinations, interpolating between the two extreme cases. Save the output.

# Imports 

In [1]:
from itertools import product

In [2]:
import numpy as np
import scipy
import matplotlib.pyplot as plt

In [3]:
import tenpy
import tenpy.linalg.np_conserved as npc
from tenpy.algorithms import dmrg
from tenpy.networks.mps import MPS

In [4]:
from tenpy.networks.terms import MultiCouplingTerms
from tenpy.networks.mpo import MPO

from tenpy.networks.site import ClockSite, FermionSite
from tenpy.models.lattice import Chain
from tenpy.models.model import CouplingMPOModel

# Define model

In [5]:
def mod_4_to_bit(x):    
    possible_mod_4_values = list(range(4))
    assert x in possible_mod_4_values

    x0 = x % 2
    x1 = ((x - x0) // 2) % 2

    return (x0, x1)

In [6]:
def bit_to_mod_4(bit_pair):
    x0, x1 = bit_pair

    possible_mod_2_values = list(range(2))
    assert x0 in possible_mod_2_values
    assert x1 in possible_mod_2_values

    return x0 + 2*x1

In [7]:
def mod_4_to_bit_addition(x, y):
    (x0, x1) = mod_4_to_bit(x)
    (y0, y1) = mod_4_to_bit(y)

    out0 = (x0 + y0) % 2
    out1 = (x1 + y1) % 2

    out = bit_to_mod_4((out0, out1))

    return out

In [8]:
def get_n1_func(n1_01, n1_10):
    l = [0, n1_01, n1_10, (n1_01 + n1_10)%2]
    
    def f(g1, g2):
        i = mod_4_to_bit_addition(g1, g2)
        return l[i]

    return f

In [9]:
n1_pair_to_fermion_operators_dict = {
    (0, 0): "Id",
    (0, 1): "C",
    (1, 0): "Cd",
    (1, 1): "N"
}

In [10]:
n1_pairs = [
    (0, 0),
    (0, 1),
    (1, 0),
    (1, 1)
]

In [11]:
pauli_proj_rep_phases = np.array([
    [1, 1, 1, 1],
    [1, 1, 1j, -1j],
    [1, -1j, 1, 1j],
    [1, 1j, -1j, 1]
])

In [12]:
def get_proj_rep_term_phase(group_quad):
    g_left, g_in, g_out, g_right = group_quad

    g1 = mod_4_to_bit_addition(g_left, g_in)
    g2 = mod_4_to_bit_addition(g_in, g_right)
    g3 = mod_4_to_bit_addition(g_left, g_out)
    g4 = mod_4_to_bit_addition(g_out, g_right)

    numerator_phase = pauli_proj_rep_phases[g1, g2]
    denominator_phase = pauli_proj_rep_phases[g3, g4]

    return numerator_phase/denominator_phase

In [13]:
def get_op_list(g_quad, n1_func):
    out = list()
    g_left, g_in, g_out, g_right = g_quad

    out.append((f'map_{g_left}_{g_left}', 0, 0))
    
    out.append((f'map_{g_in}_{g_out}', 1, 0))

    out.append((f'map_{g_right}_{g_right}', 2, 0))

    if n1_func(g_left, g_out):
        out.append(('Cd', 0, 1))
    if n1_func(g_out, g_right):
        out.append(('Cd', 1, 1))

    if n1_func(g_in, g_right):
        out.append(('C', 1, 1))
    if n1_func(g_left, g_in):
        out.append(('C', 0, 1))

    return out

In [14]:
def get_op_list_trivial(g_quad):
    out = list()
    g_left, g_in, g_out, g_right = g_quad

    out.append((f'map_{g_left}_{g_left}', 0, 0))
    
    out.append((f'map_{g_in}_{g_out}', 1, 0))

    out.append((f'map_{g_right}_{g_right}', 2, 0))

    out.append(('C', 1, 1))
    out.append(('C', 0, 1))

    out.append(('Cd', 0, 1))
    out.append(('Cd', 1, 1))

    return out

In [49]:
class ClusterFermionWithTrivial(CouplingMPOModel):
        default_lattice = "Chain"
        force_default_lattice = True

        # Would it be easier to subclass ClockSite?
        def init_sites(self, model_params):
            spin = ClockSite(4, conserve=None)
            for i in range(4):
                for j in range(4):
                    X = np.zeros((4,4))
                    X[j, i] = 1
                    op_name = f"map_{i}_{j}"
                    spin.add_op(op_name, X)
            ferm = FermionSite(conserve=None)
            sites = [spin, ferm]
            return [spin, ferm], ['s', 'f']

        def init_terms(self, model_params):
            # Read off model parameters
            n1_func = get_n1_func(1, 0)

            t = model_params.get('interpolation', 0)
            group_quads = product(range(4), repeat=4)

            for group_quad in group_quads:
                op_list_trivial = get_op_list_trivial(group_quad)
                op_list_non_trivial = get_op_list(group_quad, n1_func)

                self.add_multi_coupling((-1/4)*t, op_list_trivial)
                self.add_multi_coupling((-1/4)*(1-t), op_list_non_trivial)

# Run DMRG

In [45]:
dmrg_params = {
    "trunc_params": {"chi_max": 8, "chi_min": 1, "svd_min": 1.e-10},
    "min_sweeps":100,
    "max_sweeps":200,
    "mixer": True,
    "combine":False,
    'decay':2,
    'amplitude':10e-1,
    'disable_after':60,
    'update_env':0
}

In [46]:
import h5py
from tenpy.tools import hdf5_io

In [50]:
interpolation_values = np.linspace(0, 1 , 21)

In [51]:
interpolation_values

array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,
       0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ])

In [57]:
num_unit_cells = 100

for t in interpolation_values:
        print(f"Commencing interpolation={t} model")
        model=ClusterFermionWithTrivial({'interpolation': t, 'L': num_unit_cells})
        
        psi = MPS.from_lat_product_state(model.lat, [['0', 'empty'],]*num_unit_cells)
        psi.canonical_form()
    
        eng = dmrg.TwoSiteDMRGEngine(psi, model, dmrg_params)
        e, psi = eng.run()

        print(f"Energy: {e}\n")

        data = {
            "wavefunction": psi,
            "energy": e,
            "paramters": {"interpolation": t}
        }

        t_string = str(int(100*t))
        one_minus_t_string = str(int(100*(1-t)))

        file_name = f'{t_string}_pc_triv_{one_minus_t_string}_pc_non_triv'
    
        filename = (
            r"../../data/interpolated_trivial_to_nontrivial_fermionic_trivial_proj_rep_200_site_dmrg/{}"
            .format(file_name)
        )

        filename += ".h5"

        with h5py.File(filename, 'w') as f:

            hdf5_io.save_to_hdf5(f, data)

Commencing interpolation=0.0 model
Energy: -98.00000000000011

Commencing interpolation=0.05 model


/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


Energy: -93.72845415159874

Commencing interpolation=0.1 model


/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


Energy: -89.4944168563192

Commencing interpolation=0.15000000000000002 model


...

Energy: -89.49441687840036

Commencing interpolation=0.9500000000000001 model


/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


Energy: -93.72845415265758

Commencing interpolation=1.0 model


/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


Energy: -98.00000000000023

